In [1]:
import sys
import os
sys.path.append(os.path.abspath('../'))

import numpy as np
from random import randint
import math

from ortc.utils import *
from ortc.glop_v2 import glop_v2
from otc.exactOTC import exact_otc
from otc.exactOTC import exact_otc2
from otc.exactOTC import * 

from experiment.isomorphism import *

In [11]:
A1 = random_lollipop_fill(7, 10)
n = A1.shape[0]

# Random permutation
perm = np.random.permutation(n)
A2 = A1[np.ix_(perm, perm)]

# Get transition matrices
P1 = adj_to_trans(A1)
P2 = adj_to_trans(A2)

# Get cost function
c = get_degree_cost(A1, A2)

# Run algorithm
_, ortc_cost, ortc_weight = glop_v2(A1, A2, c, vertex=True)
ortc_alignment = np.sum(ortc_weight, axis=(2, 3))
_, otc_cost, _, otc_alignment = exact_otc(P1, P2, c)

# Get alignment
idx_ortc = np.argmax(ortc_alignment, axis=1)
idx_otc = np.argmax(otc_alignment, axis=1)

print(math.isclose(ortc_cost, 0, abs_tol=1e-9), math.isclose(otc_cost, 0, abs_tol=1e-9), check_isomorphism(idx_ortc, A1, A2), check_isomorphism(idx_otc, A1, A2))

True True True False


In [14]:
_, otc_cost2, _, otc_alignment2 = exact_otc2(P1, P2, c)
idx_otc2 = np.argmax(otc_alignment2, axis=1)
check_isomorphism(idx_otc2, A1, A2)

True

In [15]:
_, otc_cost, P, otc_alignment = exact_otc(P1, P2, c)

In [42]:
get_best_stat_dist(P, c)

(array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 

In [46]:



from scipy.linalg import solve
# Modify P for solving
A = P.T - np.eye(P.shape[0])
A[-1, :] = 1  # Replace the last row with ones to represent sum(pi) = 1

b = np.zeros(P.shape[0])
b[-1] = 1  # Right-hand side for the sum(pi) = 1 constraint

# Solve the system
stationary_distribution = solve(A, b)

In [47]:
stationary_distribution

array([ 1.19577142e-17,  4.89616321e-34,  9.70074419e-18, -1.04069290e-19,
       -0.00000000e+00, -2.44756844e-34, -4.54832002e-18,  2.05658568e-17,
       -3.01327027e-18, -1.76084393e-18,  1.12903226e-01,  2.61041617e-18,
       -1.62566540e-19, -6.81810615e-18, -0.00000000e+00,  8.07751617e-17,
        9.70047654e-19, -5.13642060e-19, -6.26245493e-20,  3.16440182e-18,
        1.08286290e-17,  9.67741935e-02,  8.82014353e-18, -6.59662626e-20,
        3.21361120e-17, -4.00726346e-17,  3.23848339e-18,  1.12414212e-17,
       -4.40914732e-18, -1.78018516e-17,  8.85474087e-17, -1.31933785e-17,
       -4.52158308e-19,  8.06451613e-02,  2.76473524e-17, -1.09637338e-18,
       -1.32625154e-20, -0.00000000e+00, -2.80442526e-21, -5.79040527e-21,
       -5.60885053e-21, -0.00000000e+00, -1.00924977e-18,  7.90050163e-17,
        2.86673565e-17,  6.01139908e-17,  3.46908711e-18, -7.15817481e-18,
        1.61051548e-19,  2.57852096e-19, -1.27999815e-17,  5.19227426e-18,
       -9.25641619e-17,  

In [37]:
n = P.shape[0]
c = c.reshape(n, -1).flatten()  

Aeq = np.vstack([P.T - np.eye(n), np.ones((1, n))])
beq = np.hstack([np.zeros(n), [1]])

# Create the linear solver with the GLOP 
solver = pywraplp.Solver.CreateSolver('GLOP')

# Decision variables: stat_dist[i]
stat_dist = [solver.NumVar(0.0, solver.infinity(), f'stat_dist_{i}') for i in range(Aeq.shape[1])]

constraints = []

# Constraints: (P' - I) * stat_dist = 0
for i in range(Aeq.shape[0]):
    constraints.append(solver.Constraint(beq[i], beq[i]))
    for j in range(Aeq.shape[1]):
        constraints[-1].SetCoefficient(stat_dist[j], Aeq[i][j])
    # solver.Add(constraint_expr == beq[i])

# Constraint: sum(stat_dist) == 1
# solver.Add(solver.Sum(stat_dist) == 1)

# Create objective function
solver.Minimize(solver.Sum(stat_dist * c))

# Solve the problem.
status = solver.Solve()
print(status)
    

2


In [35]:
if status == pywraplp.Solver.OPTIMAL:
    stat_dist_values = np.array([var.solution_value() for var in stat_dist])
    exp_cost = solver.Objective().Value()
    print(stat_dist_values, exp_cost)

[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0.] 0.0


In [19]:
n = P.shape[0]
c = c.reshape(n, -1).flatten()  

Aeq = np.vstack([P.T - np.eye(n), np.ones((1, n))])
beq = np.hstack([np.zeros(n), [1]])


# Create the linear solver with the GLOP 
solver = pywraplp.Solver.CreateSolver('GLOP')

# Decision variables: stat_dist[i]
stat_dist = [solver.NumVar(0.0, solver.infinity(), f'stat_dist_{i}') for i in range(n)]

# Constraints: (P' - I) * stat_dist = 0
for i in range(n):
    constraint_expr = solver.Sum((P[j, i] - (1.0 if i == j else 0.0)) * stat_dist[j] for j in range(n))
    solver.Add(constraint_expr == 0)

# Constraint: sum(stat_dist) == 1
solver.Add(solver.Sum(stat_dist) == 1)

# Create objective function
solver.Minimize(solver.Sum(stat_dist * c))

# Solve the problem.
status = solver.Solve()

# if status == pywraplp.Solver.OPTIMAL:
#     stat_dist_values = np.array([var.solution_value() for var in stat_dist])
#     exp_cost = solver.Objective().Value()
#         # return stat_dist_values, exp_cost
# else:
#     # In case the solver fails, try rescaling constraints.
#     alpha = 1
#     while alpha >= 1e-10:
#         alpha /= 10
#         # Create a new solver instance to reset all variables and constraints.
#         solver = pywraplp.Solver.CreateSolver('GLOP')

#         # Decision variables: stat_dist[i]
#         stat_dist = [solver.NumVar(0.0, solver.infinity(), f'stat_dist_{i}') for i in range(n)]

#         # Constraints: alpha * (P' - I) * stat_dist = 0
#         for i in range(n):
#             constraint_expr = solver.Sum(alpha * (P[j, i] - (1.0 if i == j else 0.0)) * stat_dist[j] for j in range(n))
#             solver.Add(constraint_expr == 0)

#         # Constraint: alpha * sum(stat_dist) == alpha
#         solver.Add(solver.Sum([alpha * var for var in stat_dist]) == alpha)

#         # Create objective function
#         solver.Minimize(solver.Sum(stat_dist * c))

#         # Solve the problem again.
#         status = solver.Solve()
#         if status == pywraplp.Solver.OPTIMAL:
#             stat_dist_values = np.array([var.solution_value() for var in stat_dist])
#             exp_cost = solver.Objective().Value()
#             print(alpha, stat_dist_values, exp_cost)

#     # If still no solution, raise an error.
#     # raise ValueError('Failed to compute stationary distribution.')

1.0000000000000003e-10 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0.] 0.0
1.0000000000000003e-11 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.

In [66]:
n = P.shape[0]
c = c.reshape(n, -1).flatten()  # Ensure c is a flat array of length n

Aeq = np.vstack([P.T - np.eye(n), np.ones((1, n))])
Aeq

array([[-1.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0., -1.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0., -1., ...,  0.,  0.,  0.],
       ...,
       [ 0.,  0.,  0., ...,  0., -1.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0., -1.],
       [ 1.,  1.,  1., ...,  1.,  1.,  1.]])

In [50]:
import numpy as np
from scipy.optimize import linprog

def get_best_stat_dist2(P, c):
    # Set up constraints.
    n = P.shape[0]
    c = c.reshape(n, -1).flatten()  # Ensure c is a flat array of length n

    Aeq = np.vstack([P.T - np.eye(n), np.ones((1, n))])
    beq = np.hstack([np.zeros(n), [1]])

    # Define lower bounds
    lb = np.zeros(n)
    bounds = [(lb_i, None) for lb_i in lb]  # Upper bounds are None (unbounded)

    # Define options.
    options = {'disp': False, 'presolve': False}

    # Solve linear program.
    res = linprog(c, A_eq=Aeq, b_eq=beq, bounds=bounds, method='highs', options=options)

    if res.success:
        stat_dist = res.x
        exp_cost = res.fun
    else:
        stat_dist = None
        exp_cost = None

    return stat_dist, exp_cost


In [56]:
otc_alignment3, _= get_best_stat_dist2(P, c)
otc_alignment3 = otc_alignment3.reshape(15, 15)


In [64]:
otc_alignment3

array([[ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        , -0.        ,  0.        ,
         0.11290323,  0.        ,  0.        ,  0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.09677419,  0.        ,  0.        , -0.        ,
         0.        ,  0.        , -0.        ,  0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        ,  0.08064516, -0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        , -0.        ,  0.        ,  0.        ,  0.09677419,
         0.        ,  0.        , -0.        ,  0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        , -0.        ,  0.        , 

In [57]:
idx_otc3 = np.argmax(otc_alignment3, axis=1)
check_isomorphism(idx_otc3, A1, A2)

True

In [ ]:
_, otc_cost2, _, otc_alignment2 = exact_otc2(P1, P2, c)
idx_otc2 = np.argmax(otc_alignment2, axis=1)
check_isomorphism(idx_otc2, A1, A2)

In [35]:
get_stat_dist(P)

array([-7.81621119e-16,  1.87480830e-26, -6.94141692e-16, -7.52078361e-19,
       -0.00000000e+00, -1.58370037e-26, -1.29175431e-17, -5.32081574e-16,
       -1.18478564e-16, -7.65012001e-18,  1.12903226e-01, -4.60418034e-18,
       -9.36871620e-19, -2.89184701e-16, -0.00000000e+00, -1.67947143e-16,
       -6.29305344e-16, -2.12460077e-17, -9.29457923e-19,  3.26641993e-18,
       -2.00896907e-16,  9.67741935e-02, -4.05572054e-16, -1.79686046e-18,
       -3.05004981e-18,  2.46213678e-19,  3.35703178e-18,  9.41457128e-18,
       -2.24743749e-16, -3.58985481e-16, -3.94729385e-16, -1.24971864e-16,
       -5.17882422e-16,  8.06451613e-02, -2.82434393e-18, -4.99342688e-17,
       -1.79713504e-19, -0.00000000e+00, -6.23299700e-20, -4.88068377e-20,
        1.60080936e-19, -0.00000000e+00, -2.71989318e-17, -3.39795538e-16,
       -2.80433244e-16, -1.18056807e-16, -8.59294696e-16, -1.35735454e-16,
        3.39451213e-19,  4.46469301e-19, -2.08347947e-16, -9.00297382e-18,
       -1.78580581e-16, -